In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True, encode_special_tokens=True)

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [5]:
def count_encode(inputs:str=""):
    encoded_input=tokenizer.encode(inputs)
    num_tokens=len(encoded_input)
    return encoded_input,num_tokens

def decode(inputs:list=[]):
    decode_sentence=tokenizer.convert_ids_to_tokens(inputs)
    return decode_sentence

text="here is a text demo"
encoded_input,num_tokens=count_encode(text)
print(f'Count Token:{num_tokens}')
print(f'Token IDs:{encoded_input}')
decode_sentences=decode(encoded_input)
print(f"Decode sentence:{decode_sentences}",end="\t")

Count Token:7
Token IDs:[64790, 64792, 985, 323, 260, 2254, 16948]
Decode sentence:['[gMASK]', 'sop', '▁here', '▁is', '▁a', '▁text', '▁demo']	

In [6]:
messages=[]
messages.append({
    "role":"user",
    "content":"here is a text demo"
})

chat_inputs=tokenizer.apply_chat_template(messages,add_generation_prompt=True,tokenize=True)
num_tokens=len(chat_inputs)
print(f"Count Token:{num_tokens}")
print(f"Token IDs:{chat_inputs}")
model_input_tokens=tokenizer.convert_ids_to_tokens(chat_inputs)
print(model_input_tokens,end='\t')

Count Token:11
Token IDs:[64790, 64792, 64795, 30910, 13, 985, 323, 260, 2254, 16948, 64796]
['[gMASK]', 'sop', '<|user|>', '▁', '<0x0A>', '▁here', '▁is', '▁a', '▁text', '▁demo', '<|assistant|>']	

In [12]:
from transformers import  AutoTokenizer
from zhipuai import  ZhipuAI
class ChatSession:
    def __init__(self,max_tokens=8192):
        self.client=ZhipuAI()
        self.history=[]
        self.tokenizer=AutoTokenizer.from_pretrained(
            "THUDM/chatglm3-6b",
            trust_remote_code=True,
            encode_sepcial_tokens=True
        )
        self.max_tokens=max_tokens
    
    def calculate_tokens(self,inputs):
        return len(tokenizer.apply_chat_template(
            inputs,
            add_generation_prompt=True,
            tokenize=True
        ))
    
    def chat(self):
        while True:
            user_input=input("You:")
            if user_input.lower() in ["exit","quit"]:
                break
            self.history.append({
                "role":"user",
                "content":user_input
            })
            print(f"Human:{user_input}")
            user_tokens=self.calculate_tokens(self.history)
            print("==============")
            print(f"Human message with history tokens:{user_tokens}")

            response=self.client.chat.completions.create(
                model="glm-4",
                messages=self.history,
                top_p=0.7,
                temperature=0.9,
                stream=False,
                max_tokens=self.max_tokens,
            )
            reply=response.choices[0].message.content
            print(f"AI:{reply}")
            self.history.append({
                "role":"assistant",
                "content": reply
            })
            reply_tokens=self.calculate_tokens(self.history[-2:])
            print("===================")
            print(f"AI response tokens:{reply_tokens}",end="\n*************\n")




chat_session=ChatSession()
chat_session.chat()

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


Human:你好啊
Human message with history tokens:9
AI:你好！有什么我可以帮助你的吗？
AI response tokens:21
*************
Human:没有回复?
Human message with history tokens:28
AI:很抱歉，我之前的回复没有成功发送。如果你有任何问题或需要帮助，请随时告诉我，我会尽力帮助你。
AI response tokens:39
*************
Human:你得数据到啥时候呢?
Human message with history tokens:68
AI:作为一个AI，我的数据更新是持续进行的。我旨在提供最新和最准确的信息，但请注意，我的知识库可能不包括最新的发展或事件。如果你有具体的问题或需要最新的信息，请告诉我，我会尽力提供帮助。
AI response tokens:67
*************
Human:告诉我10个关于程序员的笑话?并且用日语输出
Human message with history tokens:138
AI:当然可以，以下是10个关于程序员的笑话，用日语表达：

1. プログラマーのワンジョンは、コメントを書かないとバグが出ないと信じている。

2. あるプログラマーは、自分のコードを修正するのが面倒で、新しいプログラムを書くことにした。

3. プログラマーは、常に「これは最後のバグです」と言っているが、それは決して最後ではない。

4. プログラマーは、午後に寝ているのを「デバッグ」している。

5. あるプログラマーは、自分のPCに「Enter」キーを押すと、彼の上司が現れるように設定した。

6. プログラマーの言葉には、彼らが自分のコードを読む能力を疑っていることが分かる。

7. あるプログラマーは、自分のPCに「Esc」キーを押すと、彼の上司が消えるように設定した。

8. プログラマーは、常に自分のコードを修正するのが面倒で、他のプログラマーのコードを書き換えることにした。

9. あるプログラマーは、自分のPCに「Ctrl + Alt + Delete」を押すと、彼の上司が解雇されるように設定した。

10. プログラマーは、自分のコードを修正するのが面倒で、自分のPCを壊してしまった。
